# HW3: Transformer from Scratch

In this exercise, you are replicating character-level transformer from scratch with Pytorch Lightning. You should end up with similar code to [nanoGPT](https://github.com/karpathy/nanoGPT).

We have prepared for you a dataset and dataloader of พระอภัยมณี by สุนทรภู่ , a famous Thai poet. You should receive your very own nanoสุนทรภู่ by the end of this exercise.

Reference: [Andrej Kaparty - Let's build GPT from Scratch](https://www.youtube.com/watch?v=kCc8FmEb1nY).  
Data Source: [Vajirayana - พระอภัยมณี](https://vajirayana.org/%E0%B8%9E%E0%B8%A3%E0%B8%B0%E0%B8%AD%E0%B8%A0%E0%B8%B1%E0%B8%A2%E0%B8%A1%E0%B8%93%E0%B8%B5)

In [ ]:
!pip -q install lightning
!wget https://github.com/Knight-H/thai-lm/raw/refs/heads/master/data/pra-apai-manee-ch1-50.txt

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import lightning as L
from datetime import datetime

# hyperparameters
batch_size = 16 # B: how many independent sequences will we process in parallel?
seq_len = 256    # T: what is the maximum context length for predictions?
n_embd = 64     # C: text embedding size
n_head = 4      # number of heads
n_layer = 4     # number of blocks
max_iters = 5000
eval_interval = 250
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
dropout = 0.0
# ------------

torch.manual_seed(42)

In [ ]:
with open('pra-apai-manee-ch1-50.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("Length of dataset in characters: ", len(text))
# let's look at the first 1000 characters
print(text[:1000])

In [ ]:
# Quick implementation of character tokenizer
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"All Characters: {''.join(chars)}")
print(f"Vocab Size: {vocab_size}")

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("สวัสดีครับ"))
print(decode(encode("สวัสดีครับ")))

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

class TextDataset(torch.utils.data.Dataset):
  def __init__(self, data, seq_len):
    self.data = data
    self.seq_len = seq_len
  def __len__(self):
    return len(self.data)-seq_len
  def __getitem__(self, idx):
    return self.data[idx:idx+seq_len], self.data[idx+1:idx+seq_len+1]

train_dataset = TextDataset(train_data, seq_len)
val_dataset = TextDataset(val_data, seq_len)
print(train_dataset[0])

train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size=batch_size, shuffle=True)

## Part 1: Self-Attention Head (Scaled Dot-Product Attention)

This part implements the 3.2.1 Scaled Dot-Product Attention in the paper _Attention is All You Need_.

$$
\text{Attention}(Q,K,V) = \text{softmax}( \frac{QK^T}{\sqrt{d_k}} )V
$$

In [ ]:
B,T,C = batch_size,seq_len,n_embd # batch, time, channels
head_size = n_embd//n_head    # 16

### 1.1 Implementing Queries, Keys, Values

This should be the easiest step of the self-attention. Given $x$ with the shape of $B \times T \times C$ (batch size, time/sequence length, channel/text embedding size), multiply it with the Query, Key, and Value embedding matrix to get $q$,$k$,$v$ vectors of shape $B \times T \times d_k$. Where $d_k$ is the head size (size of each query, key, value vector).

<div>
<img src="https://jalammar.github.io/images/t/transformer_self_attention_vectors.png" width="500" />
</div>

Use `nn.Linear` to define the `key`, `query`, and `value` embedding weights (take note to not include the bias). And calculate the `k`, `q`, and `v` vectors from $x$.

In [ ]:
torch.manual_seed(42)
x = torch.randn(B,T,C)

#### FILL CODE HERE ####
# Fill in these weight matrices
key = None
query = None
value = None

# Calculate k,q,v vectors
k = None   # (B, T, d_k)
q = None   # (B, T, d_k)
v = None   # (B, T, d_k)
######################

In [ ]:
k_0_0 = torch.Tensor([0.7020,  0.6227, -0.2063, -1.0227,  0.5020, -1.4468,  0.7205, -0.3498, -0.8498,  0.9094,  0.1168, -0.9637, -0.4064, -0.0979,  1.5379, -0.0558])
q_0_0 = torch.Tensor([-1.0709, -1.1625,  0.0260,  0.3891, -0.5746,  0.1046, -0.5273,  0.1213, 1.1707,  0.2108,  0.4636,  0.3899,  1.4501, -0.0414,  0.9155,  0.0261])
print(torch.allclose(k_0_0, k[0][0].data, atol=1e-4, rtol=0))
print(torch.allclose(q_0_0, q[0][0].data, atol=1e-4, rtol=0))

### Q1: What is the first number of v[0][0]?

### 1.2 Calculate Dot Product of Query and Value

Perform dot product of `q` and `k` using `torch.matmul` or `@` such that it has shape $B \times T \times T$. Do note that `transpose` is required for this to work, since both are at shape $B \times T \times d_k$. And normalize the resulting weights `wei` by $\sqrt{d_k}$.

<div>
<img src="https://jalammar.github.io/images/t/self-attention_softmax.png" width="500" />
</div>

Please take a look at the resulting `q` and `k` dot product. In a single batch, a `q` matrix has dimensions $T \times d_k$ (each row represent the sequence length and columns are the embeddings of head size). We can view each row of `q` as the $\vec{q}_1$ query vector represented above. The dot product would represent the following resulting matrix:


$$
\begin{bmatrix} \color{red}{q_{1,1}} & \color{red}{q_{1,2}} & \color{red}{q_{1,3}} & \color{red}\cdots \\ q_{2,1} & q_{2,2} & q_{2,3} & \cdots \end{bmatrix}
\cdot
\begin{bmatrix} \color{blue}{k_{1,1}} & \color{blue}{k_{1,2}} & \color{blue}{k_{1,3}} & \color{blue}\cdots \\ k_{2,1} & k_{2,2} & k_{2,3} & \cdots \end{bmatrix}^T
=
\begin{bmatrix} \color{red}{\vec{q}_1} \cdot \color{blue}{\vec{k}_1} & \color{red}{\vec{q}_1} \cdot \vec{k}_2 \\ \vec{q}_2 \cdot \color{blue}{\vec{k}_1} & \vec{q}_2 \cdot \vec{k}_2  \end{bmatrix}
$$

The resulting matrix would have the dimensions of $T \times T$, where each row is the attention score of each word. For instance, referencing the image above, the first row is the attention scores of the first word "Thinking" compared with the other words in the sequence.

In [ ]:
#### FILL CODE HERE ####
wei =  None # (B, T, d_k) @ (B, d_k, T) ---> (B, T, T)
######################

### Q2: What shape is `wei` after the dot product of `q` and `k`?

### 1.3 Perform Masked Attention with `torch.tril`

Since we are making an autoregressive decoder-only block, it would be weird for the current token to be able to attend to future tokens. If we look at the figure above, it doesn't make any sense for the word "Thinking" to be able to see "Machines", else you already know the result to be generated. Hence, you need to "mask" these attentions.

<div>
<img src="https://jalammar.github.io/images/xlnet/transformer-decoder-block-self-attention-2.png" width="500" />
</div>

To do this, there is a special kind of matrix called triangular matrix. See the result of `torch.tril` below:

In [ ]:
torch.tril(torch.ones(T,T))[:8,:8]

Referring to the resulting $Q \cdot K$ of dimensions $T \times T$, each row index represents the time dimension, and the columns are all the other words in the sequence. For instance, when we are at the current word "Thinking" at time $t=1$, the generation of the second word should not be able to to access $\vec{q}_1 \cdot \vec{k}_1$, and not $\vec{q}_1 \cdot \vec{k}_2$ (corresponding to keys of "Machines" and "are"). This is illustrated in the matrix below:

$$\require{cancel}$$
$$
\begin{matrix} .\qquad \text{Thinking} & \text{Machines} & \text{are} \end{matrix} \\
\begin{matrix} \color{blue}{\text{Thinking}} \\ \text{Machines} \\ \text{are} \end{matrix}
\begin{bmatrix}  \color{blue}{\vec{q}_1 \cdot \vec{k}_1} & \cancel{\vec{q}_1 \cdot \vec{k}_2} & \cancel{\vec{q}_1 \cdot \vec{k}_3} \\ \vec{q}_2 \cdot \vec{k}_1 & \vec{q}_2 \cdot \vec{k}_2 & \cancel{\vec{q}_2 \cdot \vec{k}_3} \\ \vec{q}_3 \cdot \vec{k}_1 & \vec{q}_3 \cdot \vec{k}_2 & \vec{q}_3 \cdot \vec{k}_3 \end{bmatrix}
$$

This is very similar to the triangular matrix. If we are able to filter out the attentions where `tril == 0`, masked self attention will be achieved.

Use `masked_fill` on `wei` such that the resulting attention softmax is `0` just like the matrix above.  
Note: $-\infty$ or `float('-inf')` may be required.

In [ ]:
tril = torch.tril(torch.ones(T, T))
#### FILL CODE HERE ####
wei = None
######################
wei = F.softmax(wei, dim=-1)
wei[0][:8, :8]

In [ ]:
wei[0][0][1:] == 0

### 1.4 Calculate Dot Product of Softmax and Value

Should be self-explanatory. We are at the final stage of the equation to get a resulting matrix of shape $B \times T \times d_k$ (we use the same dimensions for key and value).

$$
\text{Attention}(Q,K,V) = \text{softmax}( \frac{QK^T}{\sqrt{d_k}} )V
$$

<div>
<img src="https://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png" width="500" />
</div>


In [ ]:
#### FILL CODE HERE ####
out = None
######################

### Q3: What shape is the resulting attention?

### 1.5 Putting it all together!

Now it's time to code the Attention Head `nn.Module`.
1. Initialize all the `nn.Linear` in the constructor. Use the hyperparameter `n_embd` for the embedding size.
2. Perform the self-attention calculations in the `forward` function

Note that there may be some differences in the implemented version:
- Since `tril` is not a parameter in the PyTorch module, it is registered as a `buffer` instead.
- A `dropout` is appended after the softmax for regularization

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        #### FILL CODE HERE ####
        self.key = None
        self.query = None
        self.value = None
        ######################

        self.register_buffer('tril', torch.tril(torch.ones(seq_len, seq_len)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        tril = self.tril[:T, :T] == 0
        #### FILL CODE HERE ####
        k = None                            # (B,T,d_k)
        q = None                            # (B,T,d_k)
        v = None                            # (B,T,d_k)

        # Calculate the attention scores
        wei = None                                       # Dot product of q * k & normalization (B, T, d_k) @ (B, d_k, T) -> (B, T, T)
        wei = None                                       # Use masked_fill on tril (B, T, T)
        wei = F.softmax(wei, dim=-1)                     # Apply softmax (B, T, T)
        wei = self.dropout(wei)                          # Added dropout
        out = None                                       # (B, T, T) @ (B, T, d_k) -> (B, T, d_k)
        ######################
        return out

### Q4: What is Head's output shape?

In [ ]:
x = torch.randn(B,T,C)
head = Head(head_size)
#### FILL CODE HERE ####
out = None
######################

## Part 2: Multi-Head Attention

This part implements the 3.2.2 Multi-Head Attention in the paper _Attention is All You Need`.

$$
\text{MultiHead}(Q,K,V) = \text{Concat}(\text{head}_1, ... ,\text{head}_h)W^O\\
\text{where}\: \text{head}_i = \text{Attention}(QW_i^Q,KW_i^K,VW_i^V)
$$

With multiple heads running in parallel, this would give rise to multiple representation subspaces, where each head would have multiple sets of Q/K/V matrices.


<div>
<img src="https://jalammar.github.io/images/t/transformer_attention_heads_qkv.png" width="500" />
</div>

The resulting attention would be all concatenated to a long matrix.

To preserve the shape of the vector back to the embedding size $C$ before the feed-forward layer (if the concatenation does not have the same size as the embedding), we use a projection layer $W^O$ with dimensions $hd_k \times C$.

<div>
<img src="https://jalammar.github.io/images/t/transformer_attention_heads_weight_matrix_o.png" width="500" />
</div>



There are two things you need to implement:
- The `self.heads` which is an `nn.ModuleList` of all the Attention `Head` of $h$ (`num_heads`) layers (these will be computed in parallel). And `torch.cat` to concatenante the heads in `forward`.
- The `self.proj` projection layer $W^O$ (with dimensions $C \times C$ as noted below). Use the hyperparameter `n_embd` for the embedding size. Apply the projection accordingly in `forward`.

Note that in the paper, they use $d_k = C/h = 64$. The head size is equal to the embedding size divided by the number of heads. So the higher number of heads, the lower the dimension of the head to preserve computational cost equivalent to single head.

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        #### FILL CODE HERE ####
        self.heads = None
        self.proj = None
        ######################
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        #### FILL CODE HERE ####
        out = None
        out = None
        ######################
        out = self.dropout(out)
        return out

### Q5: What is MultiHead's output shape?

In [ ]:
x = torch.randn(B,T,C)
heads = MultiHeadAttention(n_head, head_size)
#### FILL CODE HERE ####
out = None
######################

## Part 3: Feed Forward

At each block, there is a fully connected feed-forward network. Implement the 3.3 Position-wise Feed-Forward Networks in the paper _Attention is All You Need_.

$$
\text{FFN}(x) = \max(0, xW_1+b_1)W_2 + b_2
$$

This part gets the resulting matrix of shape $B \times T \times C$ from the Multi-Head Attention. The paper noted that they used an embedding size dimensionality of $C=512$ and the feed-forward inner-layer dimensionality $d_{ff}=2048$, which is pretty much $4C$.

Implement the Feed-forward equation up top with `nn.Linear` and `nn.ReLU` with the correct embedding size. Use `n_embd` for embedding size. Preserve the shape of $B \times T \times C$ in the resulting matrix.

In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            #### FILL CODE HERE ####

            ######################
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
x = torch.randn(B,T,C)
_module = FeedFoward(n_embd)
out = _module(x)
print(out.shape)
out.shape == torch.Size([B,T,n_embd])

### Q6: How many parameters are in a FeedForward module?

## Part 4: Transformer Block

Putting all the blocks together! We have initialize the constructor with all the defined previous components along with `nn.LayerNorm`.

<div>
<img src="https://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png" width="400" />
</div>
**Note: DO NOT reference this image**

Now I would like you to implement the residual connections, self-attention, feed forward, and the layer norm in `forward`. As a slight deviation from the paper, now it is more common to do pre-norm, which is to apply `LayerNorm` before self-attention.

<div>
<img src="https://drive.google.com/uc?export=view&id=1QnTkcVlyoseiZk65-IHbQhESpwmX6Zfx" width="400" />
</div>

1. Apply the first layer norm to $x$, put it through self-attention layer, and add in the residual connection.
2. Apply the second layer norm, put it through feed forward layer, and add in the residual connection.

Reference: [Andrej Kaparty - Let's build GPT from Scratch](https://www.youtube.com/watch?v=kCc8FmEb1nY).   
[Prenorm](https://arxiv.org/pdf/2002.04745)

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        #### FILL CODE HERE ####
        x = None
        x = None
        ######################
        return x

## Part 5: Language Model

In [ ]:
class TransformerLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(seq_len, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x)    # (B,T,C)
        x = self.ln_f(x)      # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -seq_len:]            # crop idx to the last block_size tokens
            logits, loss = self(idx_cond)           # get the predictions
            logits = logits[:, -1, :]               # focus only on the last time step - becomes (B, C)
            probs = F.softmax(logits, dim=-1)       # apply softmax to get probabilities - (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # sample from the distribution - (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # append sampled index to the running sequence - (B, T+1)
        return idx

In [ ]:
class TransformerLMModule(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = TransformerLanguageModel()

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        xb, yb = batch
        # evaluate the loss
        logits, loss = self.model(xb, yb)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        xb, yb = val_batch
        logits, loss = self.model(xb, yb)
        self.log('val_loss', loss, prog_bar=True)

    def on_train_batch_end(self, outputs, batch, batch_idx):
        metrics = self.trainer.callback_metrics
        if batch_idx % self.trainer.log_every_n_steps == 0:
            now = datetime.now()
            print(f'{now.strftime("%Y-%m-%dT%H:%M:%S")} Step: {batch_idx}/{self.trainer.max_steps} Train Loss: {metrics["train_loss"]:.4f}', end='')

    def on_validation_epoch_end(self):
        metrics = self.trainer.callback_metrics
        print(f'\t\t\tVal Loss: {metrics["val_loss"]:.4f}')

L.pytorch.seed_everything(42)
m = TransformerLMModule()
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters') # print the number of parameters in the model

### Q7: How many parameters are in your nanoGPT model?

In [ ]:
trainer = L.Trainer(deterministic=True, accelerator="auto", devices="auto",  logger=False, \
                    max_steps=max_iters,
                    val_check_interval = eval_interval,
                    log_every_n_steps =eval_interval,
                    enable_checkpointing =False,
                    limit_val_batches = eval_iters)
trainer.fit(m, train_dataloader, val_dataloader)

### Q8: What's the perplexity (from validation loss) on Step 5000?

### Q9: What's your output from the generated text?

In [ ]:
L.pytorch.seed_everything(42)
# generate from the model
context = torch.tensor([encode("๏ อาจารย์พีรพลสอนเอ็นแอลพี	")], dtype=torch.long, device=device)
print(decode(m.model.to(device).generate(context, max_new_tokens=1000)[0].tolist()))

## Part 6: Transformer from HuggingFace

In this part you will be using `transformers` from HuggingFace, the go-to library for many models. We will be, in similar fashion, training DistilGPT2 on the same dataset. I have provided you the tokenizer and Dataloader for ease of use.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

In [ ]:
print("The max model length is {} for this model, although the actual embedding size for GPT small is 768".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

In [ ]:
seq_len=768
batch_size=8

# Let's now split up the data into train and validation sets
n = int(0.9*len(text)) # first 90% will be train, rest val
train_text = text[:n]
val_text = text[n:]

class GPT2TextDataset(torch.utils.data.Dataset):
  def __init__(self, text, tokenizer, seq_len=768):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []
    self.seq_len = seq_len

    for i in range(0, len(text)//700):
      encodings_dict = tokenizer('<|startoftext|>'+ text[i*700:(i+1)*700] + '<|endoftext|>', truncation=True, max_length=seq_len, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

train_dataset = GPT2TextDataset(train_text, tokenizer, seq_len)
val_dataset = GPT2TextDataset(val_text, tokenizer, seq_len)
print(len(train_dataset), len(val_dataset))
print(train_dataset[0][0].shape)

train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size=batch_size, shuffle=True)

Read how Auto Classes work in HuggingFace https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes , and use `AutoModelForCausalLM.from_pretrained` to initialize your `distilgpt2` model.

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

class DistilGPT2(L.LightningModule):
    def __init__(self):
        super().__init__()
        #### FILL CODE HERE ####
        self.model = None
        ######################
        self.model.resize_token_embeddings(len(tokenizer))

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=5e-4)
        return optimizer

    def training_step(self, batch, batch_idx):
        xb, mask = batch
        # evaluate the loss
        loss, logits = self.model(xb, labels=xb, attention_mask=mask, token_type_ids=None)[:2]
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        xb, mask = val_batch
        loss, logits = self.model(xb, labels=xb, attention_mask=mask, token_type_ids=None)[:2]
        self.log('val_loss', loss, prog_bar=True)

    def on_train_batch_end(self, outputs, batch, batch_idx):
        metrics = self.trainer.callback_metrics
        if batch_idx % self.trainer.log_every_n_steps == 0:
            now = datetime.now()
            print(f'{now.strftime("%Y-%m-%dT%H:%M:%S")} Step: {batch_idx} Train Loss: {metrics["train_loss"]:.4f}', end='')

    def on_validation_epoch_end(self):
        metrics = self.trainer.callback_metrics
        print(f'\t\t\tVal Loss: {metrics["val_loss"]:.4f}')
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        beginning_text = tokenizer('<|startoftext|>'+"แต่ปางหลังยังมีกรุงกษัตริย์", return_tensors="pt").to(device)
        sample_outputs = self.model.generate(beginning_text['input_ids'], attention_mask=beginning_text["attention_mask"],pad_token_id=tokenizer.pad_token_id, do_sample=True, max_length =300, num_return_sequences=1)
        for i, sample_output in enumerate(sample_outputs):
              print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True).strip()))

L.pytorch.seed_everything(42)
distilgpt2 = DistilGPT2()
print(sum(p.numel() for p in distilgpt2.parameters())/1e6, 'M parameters') # print the number of parameters in the model

### Q10: How many parameters are in the DistilGPT2 model?

<font color='red'>Training should take around ~18 minutes</font>

In [ ]:
trainer = L.Trainer(deterministic=True, accelerator="auto", devices="auto",  logger=False, \
                    max_epochs =5,
                    log_every_n_steps =len(train_dataloader),
                    enable_checkpointing =False,
                    )
trainer.fit(distilgpt2, train_dataloader, val_dataloader)

### Q11: What's the perplexity (from validation loss) on the last step?

### Q12: What's the output from the generated text?

In [ ]:
L.pytorch.seed_everything(40)
beginning_text = tokenizer('<|startoftext|>'+"อาจารย์พีรพลสอนเอ็นแอลพี", return_tensors="pt")
output = distilgpt2.model.generate(beginning_text['input_ids'], attention_mask=beginning_text["attention_mask"],pad_token_id=tokenizer.pad_token_id, do_sample=True, max_length =500, num_return_sequences=1)
print(tokenizer.decode(output[0], skip_special_tokens=True).strip())